En aquesta pràctica treballareu amb un model de detecció d’objectes basat en PyTorch i la xarxa neuronal VGG16. L'objectiu es modificar el model per adaptar-lo a un problema de detecció d'objectes específic utilitzant un conjunt de dades simples.

![Exemple](08_Detecció/imgs/img.png)

Emprarem un *dataset* de detecció d'objectes que conté imatges d'estrelles. Aquest conjunt de dades és senzill i ideal per a practicar tècniques de detecció d'objectes. El podeu trobar a Kaggle al següent [enllaç](https://www.kaggle.com/datasets/kishanj/simple-object-detection). Per carregar aquest tipus de dataset haurem d'implementar una classe personalitzada que hereti de `torch.utils.data.Dataset`.

In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kishanj/simple-object-detection
!unzip simple-object-detection.zip -d ./stars_dataset

Ara que hem descarregat el dataset, actualitzarem la classe `EstrellesDataset` per carregar les imatges i les anotacions.

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import xml.etree.ElementTree as ET # Import the ElementTree library

from torch.utils.data import Dataset

class EstrellesDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        super().__init__()
        self.root_dir = root_dir
        self.transforms = transforms
        # Assuming images are in 'images' and annotations in 'annotations' within root_dir
        self.img_dir = os.path.join(root_dir, 'images')
        self.annotation_dir = os.path.join(root_dir, 'annotations')
        self.image_files = [f for f in os.listdir(self.img_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # Get annotation file path (assuming same name as image but with .xml extension)
        annotation_name = img_name.replace('.jpg', '.xml')
        annotation_path = os.path.join(self.annotation_dir, annotation_name)

        # Parse the XML file
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []
        for obj in root.findall('object'):
            label = obj.find('name').text
            # Assuming label 'star' corresponds to class 1
            if label == 'star':
                labels.append(1)
            else:
                labels.append(0) # Or handle other labels as needed

            bbox = obj.find('bndbox')
            x_min = int(bbox.find('xmin').text)
            y_min = int(bbox.find('ymin').text)
            x_max = int(bbox.find('xmax').text)
            y_max = int(bbox.find('ymax').text)
            boxes.append([x_min, y_min, x_max, y_max])

        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        if len(boxes) == 0:
             boxes = torch.zeros((0, 4), dtype=torch.float32)
             labels = torch.zeros((0,), dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)


        if self.transforms:
            image = self.transforms(image, target) # Assuming transforms can handle target dictionary

        return image, target

In [ ]:
# Create an instance of the dataset
dataset = EstrellesDataset(root_dir='./stars_dataset')


img, target = dataset[0]
print(f"Image shape: {img.size}")
print(f"Target: {target}")

## Preparació del model

Començarem carregant el model VGG16 preentrenat i adaptant-lo per a la detecció d'objectes. Afegirem capes addicionals per predir les caixes delimitadores (bounding boxes) i les classes dels objectes.

In [ ]:
from torch import nn
from torchvision import models, transforms

vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
backbone = vgg16.features

class VGG16ObjectDetector(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.avgpool = # TODO

        # Capes fully-connected de la VGG original
        self.flatten = nn.Flatten()
        self.fc =

        # Cap de classificació
        self.class_head =

        # Cap de regressió de bounding box
        self.bbox_head =

    def forward(self, x):
        #TODO

        return class_logits, bbox_preds

VGG16ObjectDetector(backbone=backbone)